## Setup

Requirements:


In [ ]:
!pip install pandas

In [ ]:
!pip install pandas numpy patsy plotly plotly_express nbformat

#Load Packages
import pandas as pd
import numpy as np
import patsy as pt
import plotly.express as px
import nbformat
import plotly as py

In [ ]:
#Load Data
data = pd.read_csv("amazon-purchases.csv")
survey_data = pd.read_csv("survey.csv")

# Data Pre-Processing

In [ ]:
#Add a prime purchase column to the data

#Create a list with prime days inside it
prime_days = ["2022-07-12", "2022-07-13", "2021-06-21", "2021-06-22", "2020-10-13", "2020-10-14", "2019-07-15", "2019-07-16", "2018-07-17", "2018-07-18",]

prime_purchase = []
for i in data["Order Date"]:
    if i in prime_days:
        prime_purchase.append(1)
    else:
        prime_purchase.append(0)
data["Prime Purchase"] = prime_purchase

In [ ]:
#Use prime purchase data to create a prime day customer list
prime_day_customer = []
for i in range(len(data)):
    if data["Prime Purchase"][i] == 1:
        prime_day_customer.append(data["Survey ResponseID"][i])
prime_day_customer = set(prime_day_customer)

prime_customer = []
for i in data["Survey ResponseID"]:
    if i in prime_day_customer:
        prime_customer.append(1)
    else:
        prime_customer.append(0)

data["Prime Customer"] = prime_customer

In [ ]:
# Merge the purchase data with customer surveys on 'Survey ResponseID'
merged_data = pd.merge(data, survey_data, on='Survey ResponseID', how='inner')

# Display the first few rows of the merged dataframe
print(merged_data.shape, data.shape, survey_data.shape)

In [ ]:
subset_data = merged_data[(merged_data['Q-demos-gender'] == 'Female') & (merged_data['Q-demos-age'] != '65 and older') & (merged_data['Prime Customer'] == 1) & (merged_data['Q-amazon-use-howmany']=='1 (just me!)')]

In [ ]:
subset_data['Revenue'] = subset_data['Quantity'] * subset_data['Purchase Price Per Unit']

In [ ]:
subset_data.head()

Data exploration

In [ ]:
unique_responses_by_gender = merged_data.groupby('Q-demos-gender')['Survey ResponseID'].nunique()
print(unique_responses_by_gender)

In [ ]:
subset_data['Order Date'].max()
record = subset_data[merged_data['Order Date'] == subset_data['Order Date'].max()]
record

In [ ]:
subset_data.columns

In [ ]:
# Convert 'Order Date' to datetime
subset_data.loc[:, 'Order Date'] = pd.to_datetime(subset_data['Order Date'])

# Group by 'Order Date' and sum the 'Purchase Price Per Unit'
transaction_totals = subset_data.groupby(['Order Date', 'Q-demos-gender'])['Purchase Price Per Unit'].sum().reset_index()

# Plot the time series
px.line(transaction_totals, x='Order Date', y='Purchase Price Per Unit', title='Transaction Totals Over Time')

In [ ]:
# Convert 'Order Date' to datetime
subset_data['Order Date'] = pd.to_datetime(subset_data['Order Date'])

# Extract year, month, day, and day of the week
subset_data['Year'] = subset_data['Order Date'].dt.year
subset_data['Month'] = subset_data['Order Date'].dt.month
subset_data['Day'] = subset_data['Order Date'].dt.day
subset_data['Day of Week'] = subset_data['Order Date'].dt.dayofweek

In [ ]:
subset_data.head()

# Clustering for Feature Selection

K-means for purchase quantity and purchase totals

In [ ]:
#Group by survey response id and count the number of prime purchases and sum the price per unit
prime_users = subset_data.groupby("Survey ResponseID").agg({"Prime Purchase":"sum", "Purchase Price Per Unit":"sum"})
#Left join prime purchases with survey data
print(prime_users.shape)
#Rename 

In [ ]:
!pip install scikit-learn matplotlib numpy

from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

k_data = list(zip(prime_users['Prime Purchase'], prime_users['Purchase Price Per Unit']))
inertias = []

for i in range(1,11):
    kmeans = KMeans(n_clusters=i)
    kmeans.fit(k_data)
    inertias.append(kmeans.inertia_)

plt.plot(range(1,11), inertias, marker='o')
plt.title('Elbow method')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=3)
kmeans.fit(k_data)

plt.scatter(prime_users['Prime Purchase'], prime_users['Purchase Price Per Unit'], c=kmeans.labels_)
plt.show()

In [ ]:
# Display the cluster centers
print("Cluster Centers:")
print(kmeans.cluster_centers_)

# Display the labels assigned to each data point
print("Cluster Labels:")
print(kmeans.labels_)

prime_users['cluster'] = kmeans.labels_

In [ ]:
# Ensure the 'cluster' column is present in prime_users
if 'cluster' not in prime_users.columns:
	prime_users['cluster'] = kmeans.labels_

# Merge the subset_data with prime_users to map the cluster labels
subset_data_with_clusters = pd.merge(subset_data, prime_users[['cluster']], on='Survey ResponseID', how='left')

# Display the first few rows of the resulting dataframe
subset_data_with_clusters.head()

In [ ]:
# Group by cluster and category, then sum the purchase totals
cluster_category_totals = subset_data_with_clusters.groupby(['cluster', 'Category'])['Purchase Price Per Unit'].sum().reset_index()

# Sort the totals within each cluster and select the top 5 categories
top_categories_per_cluster = cluster_category_totals.sort_values(['cluster', 'Purchase Price Per Unit'], ascending=[True, False]).groupby('cluster').head(10)

# Display the result
top_categories_per_cluster

In [ ]:
average_prices = subset_data.groupby(['Category','Prime Purchase'])['Revenue'].mean()

# Calculate the difference in mean between prime purchase 0 and 1 for each category
average_prices_diff = average_prices.unstack().diff(axis=1).iloc[:, -1].reset_index()
average_prices_diff.columns = ['Category', 'Difference']

# Display the top 40 values
top_40_differences = average_prices_diff.nlargest(40, 'Difference')
top_40_differences



In [ ]:
# Group by Category and Prime Purchase, then sum the Quantity
quantity_summed = subset_data.groupby(['Category', 'Prime Purchase'])['Quantity'].sum().unstack().reset_index()

# Calculate the difference in quantity between prime purchase 0 and 1
quantity_summed['Quantity Difference'] = quantity_summed[1] - quantity_summed[0]

# Calculate the total quantity for each category
quantity_summed['Quantity Total'] = quantity_summed[0] + quantity_summed[1]

# Merge with top_40_differences to display the results together
result = pd.merge(top_40_differences, quantity_summed[['Category', 'Quantity Difference', 'Quantity Total']], on='Category', how='left')

# Sort by 'Quantity Difference' and select the top 40
result = result.sort_values(by='Quantity Difference', ascending=False).head(40)

# Display the result
result

In [ ]:
average_prices.to_csv('average_prices.csv')

In [ ]:
subset_data_with_clusters.to_csv('subset_data_with_clusters.csv', index=False)

In [ ]:
# Group by cluster and Date and aggregate the quantity and purchase prices
aggregated_data = subset_data.groupby(['Order Date']).agg({
    'Quantity': 'sum',
    'Revenue': 'sum'
}).reset_index()

#Add a column for the year, month, day, and day of the week
aggregated_data['Year'] = aggregated_data['Order Date'].dt.year
aggregated_data['Month'] = aggregated_data['Order Date'].dt.month
aggregated_data['Day'] = aggregated_data['Order Date'].dt.day
aggregated_data['Day of Week'] = aggregated_data['Order Date'].dt.dayofweek

# Display the first few rows of the aggregated data
aggregated_data.head()

Clustering for dimension reduction

# Specify and Prepare the Model Type

In [ ]:
#Exponential Smoothing
from statsmodels.tsa.api import ExponentialSmoothing
from statsmodels.tsa.api import SimpleExpSmoothing
import plotly_express as px

px.line(aggregated_data, x='Order Date', y='Revenue')

In [ ]:
revenue = aggregated_data['Revenue']
revenue.index = aggregated_data['Order Date']
revenue.index.freq = revenue.index.inferred_freq

alpha020 = SimpleExpSmoothing(revenue).fit(
                                        smoothing_level=0.2,
                                        optimized=False)

alpha050 = SimpleExpSmoothing(revenue).fit(
                                        smoothing_level=0.5,
                                        optimized=False)

alpha080 = SimpleExpSmoothing(revenue).fit(
                                        smoothing_level=0.8,
                                        optimized=False)

forecast020 = alpha020.forecast(3)
forecast050 = alpha050.forecast(3)
forecast080 = alpha080.forecast(3)

In [ ]:
import plotly.graph_objects as go

# Plotting our data

smoothData = pd.DataFrame([revenue.values, alpha020.fittedvalues.values,  alpha050.fittedvalues.values,  alpha080.fittedvalues.values]).T
smoothData.columns = ['Truth', 'alpha=0.2', 'alpha=0.5', 'alpha=0.8']
smoothData.index = revenue.index

fig = px.line(smoothData, y = ['Truth', 'alpha=0.2', 'alpha=0.5', 'alpha=0.8'],
        x = smoothData.index,
        color_discrete_map={"Truth": 'blue',
                           'alpha=0.2': 'red',
                            'alpha=0.5':'green',
                            'alpha=0.8':'purple'}
       )

# Incorporating the Forecasts

fig.add_trace(go.Scatter(x=forecast020.index, y = forecast020.values, name='Forecast alpha=0.2', line={'color':'red'}))
fig.add_trace(go.Scatter(x=forecast050.index, y = forecast050.values, name='Forecast alpha=0.5', line={'color':'green'}))
fig.add_trace(go.Scatter(x=forecast080.index, y = forecast080.values, name='Forecast alpha=0.8', line={'color':'purple'}))

In [ ]:

#GAMs
!pip install pygam

from pygam import LinearGAM, s, f
import statsmodels.api as sm
import statsmodels.tsa.stattools as st
import plotly.express as px

In [ ]:
aggregated_data.rename(columns={'Day of Week': 'Weekday', 'Purchase Price Per Unit': 'Revenue'}, inplace=True)
aggregated_data.columns

In [ ]:
aggregated_data.head()

In [ ]:
from sklearn.model_selection import train_test_split

x = aggregated_data[["Year", "Month", "Day"]]
y = aggregated_data["Revenue"]

# Split into training and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
# Define the GAM model with smooth terms for each feature
gam = LinearGAM(s(0) + s(1) + s(2)).fit(x_train, y_train)

# Display model summary
print(gam.summary())

from sklearn.metrics import mean_squared_error

# Make predictions on the test set
y_pred = gam.predict(x_test)

# Evaluate model performance
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Plot smooth terms
for i, term in enumerate(["Year", "Month", "Day"]):
    plt.figure()
    plt.title(f"Smooth Term for {term}")
    gam.partial_dependence(term=i, X=x_test, width=0.95)
    plt.show()

import numpy as np

# Perform grid search for optimal smoothness
gam.gridsearch(x_train, y_train, lam=np.logspace(-3, 3, 11))

# Display the updated summary
print(gam.summary())



In [ ]:
from pygam import LinearGAM

# Assuming gam is your trained LinearGAM model
gam = LinearGAM().fit(x, y)

# Plot partial dependence
gam.summary()  # Summary should list all feature splines

# Plot each term (feature)
import matplotlib.pyplot as plt
plt.figure()
fig, axs = plt.subplots(nrows=3, figsize=(15, 15))  # Adjust nrows to the number of features

for i, ax in enumerate(axs):
    XX = gam.generate_X_grid(term=i)
    ax.plot(XX[:, i], gam.partial_dependence(term=i, X=XX))
    ax.plot(XX[:, i], gam.partial_dependence(term=i, X=XX, width=.95)[1], c='r', ls='--')
    ax.set_title(f"Partial dependence for term {i}")
plt.show()

import matplotlib.pyplot as plt

plt.plot([0, 1], [0, 1])
plt.show()

XX = gam.generate_X_grid(term=0)
print(XX)
pd = gam.partial_dependence(term=0, X=XX)
plt.plot(XX[:, 0], pd)
plt.show()



In [ ]:
from pygam import LinearGAM

# Fit the model without the third feature
gam = LinearGAM().fit(x.iloc[:, :2], y)
gam.summary()


In [ ]:
import matplotlib.pyplot as plt

# Residual plot
plt.scatter(y, gam.deviance_residuals(x.iloc[:, :2], y))
plt.xlabel('Observed')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.show()


In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Select only the first two columns of x_test
x_test_selected = x_test.iloc[:, :2]

y_pred = gam.predict(x_test_selected)
print("R^2:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", mean_squared_error(y_test, y_pred) ** 0.5)


In [ ]:
import matplotlib.pyplot as plt
for i, term in enumerate(gam.terms):
    if term.isintercept:
        continue
    XX = gam.generate_X_grid(term=i)
    plt.plot(XX[:, i], gam.partial_dependence(term=i, X=XX))
    plt.title(f"Partial Dependence of Feature {i}")
    plt.show()


In [ ]:
from pygam import LinearGAM
import numpy as np

# Perform grid search for optimal smoothness
gam = LinearGAM().gridsearch(x.values, y.values, lam=np.logspace(-3, 3, 11))
gam.summary()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Generate dates for the next 3 months
future_dates = pd.date_range(start="2024-01-01", end="2024-03-31", freq="D")

# Generate the same features as used in the training data
future_features = pd.DataFrame({
    "Year": future_dates.year,
    "Month": future_dates.month,
    "Day": future_dates.day
})

# Assume `gam` is your trained LinearGAM model
future_predictions = gam.predict(future_features)

# Get confidence intervals
intervals = gam.prediction_intervals(future_features, width=0.95)
lower = intervals[:, 0]  # Lower bounds
upper = intervals[:, 1]  # Upper bounds

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(future_dates, future_predictions, label="Predicted Revenue", color="blue")
plt.fill_between(
    future_dates, lower, upper, color="gray", alpha=0.2, label="Confidence Interval"
)
plt.xlabel("Date")
plt.ylabel("Revenue")
plt.title("Predicted Daily Revenue for the Next 3 Months")
plt.legend()
plt.grid()
plt.show()

# Save predictions to a CSV
predictions_df = pd.DataFrame({
    "date": future_dates,
    "predicted_revenue": future_predictions,
    "lower_bound": lower,
    "upper_bound": upper
})
predictions_df.to_csv("revenue_predictions.csv", index=False)



In [ ]:
#SARIMAX


# Train Models

In [ ]:
#Decision Tree

# Create the model and fit it
clf = DecisionTreeClassifier(max_depth=5)
clf.fit(x, y)

# Prediction & Validation

In [ ]:
#Decision Tree "clf"

print("\n\nIn-sample accuracy: %s%%\n\n" 
 % str(round(100*accuracy_score(y, clf.predict(x)), 2)))
print("\n\nOut-of-sample accuracy: %s%%\n\n"
%str(round(100*accuracy_score(yt, clf.predict(xt)), 2)))